In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [3]:
#Load the csv exported in Part I to a DataFrame
df = pd.read_csv('../city_weather_data.csv')
df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Okmulgee,35.62,-95.96,57.2,100.0,90.0,8.05,US,1.605358e+09
1,Punta Arenas,-53.15,-70.92,41.0,63.0,75.0,20.80,CL,1.605359e+09
2,Sault Ste. Marie,46.52,-84.33,30.2,74.0,90.0,4.70,CA,1.605359e+09
3,Puerto Ayora,-0.74,-90.35,70.0,85.0,98.0,5.01,EC,1.605359e+09
4,Rovaniemi,66.50,25.72,33.8,100.0,90.0,9.17,FI,1.605359e+09


In [5]:
#Humidity Heatmap
#Configure gmaps.

fig = gmaps.figure()

In [6]:

#Use the Lat and Lng as locations and Humidity as the weight.
#Add Heatmap layer to map.gmaps.configure(api_key=g_key)


locations = df[['Lat', 'Lng']]
weights = df['Humidity']
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
weather_criteria_df = df.loc[(df['Max Temp'] > 70) & (df['Max Temp'] < 80) & (df['Cloudiness'] == 0) & (df['Wind Speed'] < 10)].copy()
weather_criteria_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
33,Awjilah,29.11,21.29,77.07,36.0,0.0,7.92,LY,1.605359e+09
81,Sabha,27.04,14.43,74.05,31.0,0.0,7.85,LY,1.605359e+09
97,Broken Hill,-31.95,141.43,75.20,22.0,0.0,5.82,AU,1.605359e+09
119,Riyadh,24.69,46.72,77.00,25.0,0.0,4.70,SA,1.605359e+09
134,Hervey Bay,-25.30,152.85,77.00,83.0,0.0,5.99,AU,1.605359e+09
165,Coahuayana Viejo,18.73,-103.68,71.71,86.0,0.0,3.20,MX,1.605359e+09
194,Jalu,29.03,21.55,77.36,36.0,0.0,8.97,LY,1.605359e+09
208,Nālūt,30.33,10.85,72.12,27.0,0.0,1.68,LY,1.605359e+09
221,Jaguaruna,-28.61,-49.03,79.25,63.0,0.0,4.68,BR,1.605359e+09
256,Mackay,-21.15,149.20,77.00,88.0,0.0,6.93,AU,1.605359e+09


In [19]:
#Hotel Map
#Store into variable named hotel_df.

hotel_df = df.loc[(df['Max Temp'] > 70) & (df['Max Temp'] < 80) & (df['Cloudiness'] == 0) & (df['Wind Speed'] < 10)].copy()
hotel_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
33,Awjilah,29.11,21.29,77.07,36.0,0.0,7.92,LY,1.605359e+09
81,Sabha,27.04,14.43,74.05,31.0,0.0,7.85,LY,1.605359e+09
97,Broken Hill,-31.95,141.43,75.20,22.0,0.0,5.82,AU,1.605359e+09
119,Riyadh,24.69,46.72,77.00,25.0,0.0,4.70,SA,1.605359e+09
134,Hervey Bay,-25.30,152.85,77.00,83.0,0.0,5.99,AU,1.605359e+09


In [23]:
hotel_df.count()


City          20
Lat           20
Lng           20
Max Temp      20
Humidity      20
Cloudiness    20
Wind Speed    20
Country       20
Date          20
dtype: int64

In [28]:
#Add a "Hotel Name" column to the DataFrame.
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.
#Plot markers on top of the heatmap.


hotel_df['Hotel Name'] = ''
locations = hotel_df[['Lat', 'Lng']]
hotel_df = hotel_df.reset_index(drop=True)
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Awjilah,29.11,21.29,77.07,36.0,0.0,7.92,LY,1.605359e+09,
1,Sabha,27.04,14.43,74.05,31.0,0.0,7.85,LY,1.605359e+09,
2,Broken Hill,-31.95,141.43,75.20,22.0,0.0,5.82,AU,1.605359e+09,
3,Riyadh,24.69,46.72,77.00,25.0,0.0,4.70,SA,1.605359e+09,
4,Hervey Bay,-25.30,152.85,77.00,83.0,0.0,5.99,AU,1.605359e+09,


In [25]:

base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {
    'rankby': 'distance',
    'type': 'lodging',
    'key': g_key,
    'keyword': 'hotel'
}

for index, row in hotel_df.iterrows():
    location = f'{row["Lat"]},{row["Lng"]}'
    params['location'] = location
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except(SyntaxError, IndexError):
        print('sorry, no luck')

sorry, no luck
sorry, no luck


In [31]:
hotel_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Awjilah,29.11,21.29,77.07,36.0,0.0,7.92,LY,1.605359e+09,
1,Sabha,27.04,14.43,74.05,31.0,0.0,7.85,LY,1.605359e+09,
2,Broken Hill,-31.95,141.43,75.20,22.0,0.0,5.82,AU,1.605359e+09,
3,Riyadh,24.69,46.72,77.00,25.0,0.0,4.70,SA,1.605359e+09,
4,Hervey Bay,-25.30,152.85,77.00,83.0,0.0,5.99,AU,1.605359e+09,
